In [ ]:
import numpy as np
import pandas as pd
import cv2

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import sys
from tqdm import tqdm
from time import time
import shutil

In [ ]:
import matplotlib.pyplot as plt

def plot_progress(history):
    loss_tr = []
    loss_vl = []
    acc_tr = []
    acc_vl = []
    loss_tr.append(history.history['loss'])
    loss_vl.append(history.history['val_loss'])
    acc_tr.append(history.history['categorical_accuracy'])
    acc_vl.append(history.history['val_categorical_accuracy'])
    
    plt.plot(acc_tr[0])
    plt.plot(acc_vl[0])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    plt.plot(loss_tr[0])
    plt.plot(loss_vl[0])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

In [ ]:
from sklearn.metrics import classification_report,multilabel_confusion_matrix

In [ ]:
inception1 = 0
def report(model):
    prediction = model.predict(test_gen)
    if model == inception1:
        print('incept')
        prediction = prediction[0]
    predicted_labels = np.argmax(prediction, axis=1)
    confusion_mat = multilabel_confusion_matrix(test_gen.labels, predicted_labels, labels=[0,1,2])
    print(confusion_mat)
    print(classification_report(test_gen.labels, predicted_labels, labels=[0,1,2], target_names=['faulty','normal','vfaulty'], digits=4))

In [ ]:
train_path = 'Image_Data/train_dir'
val_path = 'Image_Data/eval_dir'
test_path = 'Image_Data/test_dir'

num_train_samples = 14400
num_test_samples = 3600
num_val_samples = 6000
train_batch_size = 8
val_batch_size = 8


train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

In [ ]:
IMAGE_SIZE = 256
def preprocess(img):
    img = img[len(img[0])//2:,:]
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    return img

datagen = ImageDataGenerator(preprocessing_function=lambda img: preprocess(img),
                             rescale=1.0/255)

train_gen = datagen.flow_from_directory(train_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=train_batch_size,
                                        class_mode='categorical')

val_gen = datagen.flow_from_directory(val_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=val_batch_size,
                                        class_mode='categorical')

test_gen = datagen.flow_from_directory(test_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=1,
                                        class_mode='categorical',
                                        shuffle=False)

# DL Models

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, DepthwiseConv2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras import Model

In [ ]:
# Metrics
cat_acc = tf.keras.metrics.CategoricalAccuracy()
auc_roc = tf.keras.metrics.AUC(multi_label=True, num_labels=3)
auc_pr = tf.keras.metrics.AUC(curve='PR', multi_label=True, num_labels=3)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

## AlexNet

In [ ]:
def AlexNet(input_shape):
    input = Input(shape=input_shape)
    
    x = Conv2D(96, (11, 11), strides=(4, 4), activation='relu')(input)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = Conv2D(256, kernel_size=5, padding='same', activation='relu')(x)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = Conv2D(384, kernel_size=3, padding='same', activation='relu')(x)
    x = tf.keras.layers.SpatialDropout2D(0.5)(x)
    x = BatchNormalization()(x)
    x = Conv2D(384, kernel_size=3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = BatchNormalization()(x)
    
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(3, activation='softmax')(x)
    
    model = Model(inputs=input, outputs=output, name='AlexNet')
    
    return model

In [ ]:
alexnet = AlexNet(input_shape=(256,256,3))
alexnet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,auc_pr])

In [ ]:
alexnet.count_params()

In [ ]:
start = time()
alexnet_hist = alexnet.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=50, verbose=2)
end = time()
print("===================================")
print(f'50 epochs in {(end-start)/60} min')

In [ ]:
hist_df = pd.DataFrame(alexnet_hist.history)
hist_json_file = 'AlexNet_10ep_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [ ]:
alexnet.save_weights('AlexNet.hdf5')

In [ ]:
alexnet.evaluate(test_gen)

In [ ]:
report(alexnet)

## VGG

In [ ]:
def conv_2(x,filters,kernel_size,activation):
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
    return x

def conv_3(x,filters,kernel_size,activation):
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
    return x

def conv_4(x,filters,kernel_size,activation):
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = Conv2D(filters, kernel_size=kernel_size, padding='same', activation=activation)(x)
    x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
    return x

def FC(x,units,activation):
    x = Flatten()(x) 
    x = Dense(units = units, activation =activation)(x)
    x = Dropout(0.5)(x)
    x = Dense(units = units, activation =activation)(x)
    x = Dropout(0.5)(x)
    x = Dense(units = 3, activation ='softmax')(x)
    return x

### VGG16

In [ ]:
def VGG16(input_shape):
    
    input = Input(shape=input_shape)
    
    filters = 64
    kernel_same = 3
    act = 'relu'
    
    x = conv_2(input,filters,kernel_same,act)
    x = conv_2(x,filters*2,kernel_same,act)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = conv_3(x,filters*4,kernel_same,act)
    x = conv_3(x,filters*8,kernel_same,act)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = conv_3(x,filters*8,kernel_same,act)
    
    units = 4096
    
    output = FC(x,units,act)
    
    model = Model(inputs=input, outputs=output, name='VGG16')
    
    return model

In [ ]:
vgg16 = VGG16(input_shape=(256,256,3))
vgg16.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,auc_pr])

In [ ]:
vgg16.count_params()

In [ ]:
start = time()
vgg16_hist = vgg16.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10, verbose=2)
end = time()
print("===================================")
print(f'10 epochs in {(end-start)/60} min')

In [ ]:
hist_df = pd.DataFrame(vgg16_hist.history)
hist_json_file = 'VGG16_10ep_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [ ]:
vgg16.evaluate(test_gen)

In [ ]:
report(vgg16)

### VGG19

In [ ]:
def VGG19(input_shape):
    
    input = Input(shape=input_shape)
    
    filters = 64
    kernel_same = 3
    act = 'relu'
    
    x = conv_2(input,filters,kernel_same,act)
    x = conv_2(x,filters*2,kernel_same,act)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = conv_4(x,filters*4,kernel_same,act)
    x = tf.keras.layers.SpatialDropout2D(0.5)(x)
    x = conv_4(x,filters*8,kernel_same,act)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = conv_4(x,filters*8,kernel_same,act)
    
    units = 4096
    
    output = FC(x,units,act)
    
    model = Model(inputs=input, outputs=output)
    
    return model

In [ ]:
vgg19 = VGG19(input_shape=(256,256,3))
vgg19.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,auc_pr])

In [ ]:
vgg19.count_params()

In [ ]:
start = time()
vgg19_hist = vgg19.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10, verbose=2)
end = time()
print("===================================")
print(f'10 epochs in {(end-start)/60} min')

In [ ]:
hist_df = pd.DataFrame(vgg19_hist.history)
hist_json_file = 'VGG19_10ep_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [ ]:
pred = vgg19.evaluate(test_gen)

In [ ]:
report(vgg19)

## MobileNet

In [ ]:
def block(x, filters, strides, activation):
    
    x = DepthwiseConv2D(kernel_size=3, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation)(x)
    
    x = Conv2D(filters=filters, kernel_size=1, strides=1)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation)(x)
    
    return x

def MobileNet(input_shape):
    
    activation = 'relu'
    filters = 32
    reps = 5
    input = Input(shape =input_shape)
    x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(input)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation)(x)
    
    x = block(x, filters = filters, strides = 1, activation=activation)
    x = block(x, filters = filters*2, strides = 2, activation=activation)
    x = block(x, filters = filters*2, strides = 1, activation=activation)
    x = block(x, filters = filters*4, strides = 2, activation=activation)
    x = Dropout(0.2)(x)
    x = block(x, filters = filters*4, strides = 1, activation=activation)
    x = block(x, filters = filters*8, strides = 2, activation=activation)
    for _ in range (reps):
         x = block(x, filters = filters*8, strides = 1, activation=activation)
    x = block(x, filters = filters*16, strides = 2, activation=activation)
    x = block(x, filters = filters*16, strides = 1, activation=activation)
    x = Dropout(0.2)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    output = Dense(units = 3, activation = 'softmax')(x)
    model = Model(inputs=input, outputs=output)
    
    return model

In [ ]:
mobilenet = MobileNet(input_shape=(256,256,3))
mobilenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,auc_pr])

In [ ]:
mobilenet.count_params()

In [ ]:
start = time()
mobilenet_hist = mobilenet.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10, verbose=2)
end = time()
print("===================================")
print(f'10 epochs in {(end-start)/60} min')

In [ ]:
hist_df = pd.DataFrame(mobilenet_hist.history)
hist_json_file = 'MobileNet_10ep_histori.json'
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [ ]:
pred = mobilenet.evaluate(test_gen)

In [ ]:
report(mobilenet)

## MobileNet v2

In [ ]:
def inverted_residual_block(x, factor, filters_in, filters_out, strides, activation):
    
    x_add = x
    x = Conv2D(filters=filters_in*factor, kernel_size=1, strides=1)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation)(x)
    
    x = DepthwiseConv2D(kernel_size=3, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation)(x)
    
    x = Conv2D(filters=filters_out, kernel_size=1, strides=1,use_bias=False)(x)
    x = BatchNormalization()(x)
    
    if x_add.shape[-1] == x.shape[-1]:
        x = tf.keras.layers.Add()([x_add,x])
    
    return x

In [ ]:
def MobileNet_v2(input_shape=(256,256,3)):
    
    input = Input(shape=input_shape)
    
    x = Conv2D(filters=32, kernel_size=3, strides=2, padding='same', use_bias=False)(input)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu6')(x)
    
    x = inverted_residual_block(x, factor=1, filters_in=x.shape[-1], filters_out=16, strides=1, activation='relu6')
    x = Dropout(0.2)(x)
    for i in range(2):
        s = 1
        if i == 0:
            s = 2
        x = inverted_residual_block(x, factor=6, filters_in=x.shape[-1], filters_out=24, strides=s, activation='relu6')
    for i in range(3):
        s = 1
        if i == 0:
            s = 2
        x = inverted_residual_block(x, factor=6, filters_in=x.shape[-1], filters_out=32, strides=s, activation='relu6')
    for i in range(4):
        s = 1
        if i == 0:
            s = 2
        x = inverted_residual_block(x, factor=6, filters_in=x.shape[-1], filters_out=64, strides=s, activation='relu6')
    for i in range(3):
        x = inverted_residual_block(x, factor=6, filters_in=x.shape[-1], filters_out=96, strides=1, activation='relu6')
    for i in range(3):
        s = 1
        if i == 0:
            s = 2
        x = inverted_residual_block(x, factor=6, filters_in=x.shape[-1], filters_out=160, strides=s, activation='relu6')
    x = inverted_residual_block(x, factor=6, filters_in=x.shape[-1], filters_out=320, strides=1, activation='relu6')
    x = Dropout(0.2)(x)
    
    x = Conv2D(filters=1280, kernel_size=1, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu6')(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    
    x = Flatten()(x)
    output = Dense(units=3, activation='softmax')(x)
    
    model = Model(inputs=input, outputs=output)
    
    return model

In [ ]:
mobilenet_v2 = MobileNet_v2(input_shape=(256,256,3))
mobilenet_v2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,auc_pr])

In [ ]:
mobilenet_v2.count_params()

In [ ]:
start = time()
mobilenet_v2_hist = mobilenet_v2.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10, verbose=2)
end = time()
print("===================================")
print(f'10 epochs in {(end-start)/60} min')

In [ ]:
hist_df = pd.DataFrame(mobilenet_v2_hist.history)
hist_json_file = 'MobileNe_v2_10ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [ ]:
pred = mobilenet_v2.evaluate(test_gen)

In [ ]:
report(mobilenet_v2)

## MobileNet v3

In [ ]:
def bottleneck_block(x, kernel_size, stride, filters_out, exp_size, SE, NL):

    expanded_channels = exp_size
    reduction_ratio = 4
    x_add = x
    
    x = Conv2D(expanded_channels, kernel_size=1, padding='valid')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(NL)(x)
    
    x = DepthwiseConv2D(kernel_size=kernel_size, padding='same', strides=stride)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(NL)(x)
    
    if SE == True:
        # Squeeze and Excitation Layer
        squeeze = tf.keras.layers.GlobalAveragePooling2D()(x)
        excitation = Dense(expanded_channels // reduction_ratio, activation='relu')(squeeze)
        excitation = Dense(expanded_channels, activation='hard_sigmoid')(excitation)
        excitation = tf.keras.layers.Reshape((1, 1, expanded_channels))(excitation)
        x = tf.keras.layers.multiply([x, excitation])

    x = Conv2D(filters_out, kernel_size=1, padding='valid')(x)
    x = BatchNormalization()(x)
    
    if x_add.shape == x.shape:
        x = tf.keras.layers.add([x, x_add])
    
    return x

def efficient_final_layers(x,l=960):
    x = Conv2D(l, kernel_size=1, padding='valid', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation('swish')(x)
    
    x = tf.keras.layers.AveragePooling2D(8)(x)
    
    x = Conv2D(1280, kernel_size=1, use_bias=False)(x)
    x = tf.keras.layers.Activation('swish')(x)
    x = Dropout(0.2)(x)
    x = Conv2D(3, kernel_size=1, activation='softmax')(x)
    
    x = tf.squeeze(x, 1)
    x = tf.squeeze(x, 1)
    
    return x

In [ ]:
def MobileNet_v3(size='Large',input_shape=(256,256,3)):
    
    input = Input(shape=input_shape)
    
    if size == 'Large':
        x = Conv2D(16, kernel_size=3, padding='same', strides=2, use_bias=False)(input)
        x = BatchNormalization()(x)
        x = tf.keras.layers.Activation('swish')(x)

                           # x  k  s  out exp sq&ex   act
        x = bottleneck_block(x, 3, 1, 16, 16, False, 'relu')
        x = bottleneck_block(x, 3, 2, 24, 64, False, 'relu')
        x = bottleneck_block(x, 3, 1, 24, 72, False, 'relu')
        x = Dropout(0.2)(x)

        x = bottleneck_block(x, 5, 2, 40, 72, True, 'relu')
        x = bottleneck_block(x, 5, 1, 40, 120, True, 'relu')
        x = bottleneck_block(x, 5, 1, 40, 120, True, 'relu')

        x = bottleneck_block(x, 3, 2, 80, 240, False, 'swish')
        x = bottleneck_block(x, 3, 1, 80, 200, False, 'swish')
        x = bottleneck_block(x, 3, 1, 80, 184, False, 'swish')
        x = bottleneck_block(x, 3, 1, 80, 184, False, 'swish')
        x = bottleneck_block(x, 3, 1, 112, 480, True, 'swish')
        x = bottleneck_block(x, 3, 1, 112, 672, True, 'swish')
        x = Dropout(0.2)(x)

        x = bottleneck_block(x, 5, 2, 160, 672, True, 'swish')
        x = bottleneck_block(x, 5, 1, 160, 960, True, 'swish')
        x = bottleneck_block(x, 5, 1, 160, 960, True, 'swish')
        

        output = efficient_final_layers(x)

        model = Model(inputs=input, outputs=output, name='MobileNetv3_large')
    if size == 'Small':
        x = Conv2D(16, kernel_size=3, padding='same', strides=2, use_bias=False)(input)
        x = BatchNormalization()(x)
        x = tf.keras.layers.Activation('swish')(x)

                           # x  k  s  out exp sq&ex   act
        x = bottleneck_block(x, 3, 2, 16, 16, True, 'relu')
        x = bottleneck_block(x, 3, 2, 24, 72, False, 'relu')
        x = bottleneck_block(x, 3, 1, 24, 88, False, 'relu')
        x = Dropout(0.2)(x)
        
        x = bottleneck_block(x, 5, 2, 40, 96, True, 'swish')
        x = bottleneck_block(x, 5, 1, 40, 240, True, 'swish')
        x = bottleneck_block(x, 5, 1, 40, 240, True, 'swish')
        x = bottleneck_block(x, 5, 1, 48, 120, True, 'swish')
        x = bottleneck_block(x, 5, 1, 48, 144, True, 'swish')
        x = bottleneck_block(x, 5, 2, 96, 288, True, 'swish')
        x = Dropout(0.2)(x)
        x = bottleneck_block(x, 5, 1, 96, 576, True, 'swish')
        x = bottleneck_block(x, 5, 1, 96, 576, True, 'swish')

        output = efficient_final_layers(x,l=576)

        model = Model(inputs=input, outputs=output, name='MobileNetv3_Small')
    
    return model

In [ ]:
Lmobilenet_v3 = MobileNet_v3(size='Large',input_shape=(256,256,3))
Lmobilenet_v3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,auc_pr])

In [ ]:
Lmobilenet_v3.count_params()

In [ ]:
start = time()
Lmobilenet_v3_hist = Lmobilenet_v3.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10, verbose=2)
end = time()
print("===================================")
print(f'10 epochs in {(end-start)/60} min')

In [ ]:
hist_df = pd.DataFrame(Lmobilenet_v3_hist.history)
hist_json_file = 'LMobileNe_v3_10ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [ ]:
pred = Lmobilenet_v3.evaluate(test_gen)

In [ ]:
report(Lmobilenet_v3)

In [ ]:
Smobilenet_v3 = MobileNet_v3(size='Small',input_shape=(256,256,3))
Smobilenet_v3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,auc_pr])

In [ ]:
Smobilenet_v3.count_params()

In [ ]:
start = time()
Smobilenet_v3_hist = Smobilenet_v3.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10, verbose=2)
end = time()
print("===================================")
print(f'10 epochs in {(end-start)/60} min')

In [ ]:
hist_df = pd.DataFrame(Smobilenet_v3_hist.history)
hist_json_file = 'SMobileNe_v3_10ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [ ]:
pred = Smobilenet_v3.evaluate(test_gen)

In [ ]:
report(Smobilenet_v3)

## ResNet under 50

In [ ]:
def identity_block(x, filter):
    x_skip = x
    
    act = 'relu'
    
    x = Conv2D(filters=filter, kernel_size=(3,3), padding='same')(x)
    x = BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation(act)(x)
    
    x = Conv2D(filters=filter, kernel_size=(3,3), padding='same')(x)
    x = BatchNormalization(axis=3)(x)
    
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation(act)(x)
    
    return x

def convolutional_block(x, filter):
    x_skip = x
    
    act = 'relu'
    
    x = Conv2D(filters=filter, kernel_size=(3,3), padding='same', strides=2)(x)
    x = BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation(act)(x)
    
    x = Conv2D(filters=filter, kernel_size=(3,3), padding='same')(x)
    x = BatchNormalization(axis=3)(x)
    
    x_skip = Conv2D(filters=filter, kernel_size=1, strides=2)(x_skip)
    
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation(act)(x)
    return x

In [ ]:
def ResNet(layers=18, input_shape=(256,256,3)):
    
    filters = 64
    act = 'relu'
    
    input = Input(shape=input_shape)
    #x = tf.keras.layers.ZeroPadding2D((3,3))(x_input)
    
    x = Conv2D(filters=filters, kernel_size=(7,7), strides=2)(input)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(act)(x)
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    
    if layers == 18:
        block_layers = [2,2,2,2]
    if layers == 34:
        block_layers = [3,4,6,3]
    for i in range(4):
        if i == 0:
            for j in range(block_layers[i]):
                x = identity_block(x, filters)
        else:
            filters = filters*2
            x = convolutional_block(x, filters)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filters)
    
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    output = Dense(units=3, activation='softmax')(x)
    
    model = Model(inputs=input, outputs=output, name='ResNet18')
    
    return model

In [ ]:
resnet18 = ResNet(input_shape=(256,256,3))
resnet18.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,auc_pr])

In [ ]:
resnet18.count_params()

In [ ]:
start = time()
resnet18_hist = resnet18.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10, verbose=2)
end = time()
print("===================================")
print(f'10 epochs in {(end-start)/60} min')

In [ ]:
hist_df = pd.DataFrame(resnet18_hist.history)
hist_json_file = 'ResNet18_10ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [ ]:
pred = resnet18.evaluate(test_gen)

In [ ]:
report(resnet18)

In [ ]:
resnet34 = ResNet(layers=34,input_shape=(256,256,3))
resnet34.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,auc_pr])

In [ ]:
resnet34.count_params()

In [ ]:
start = time()
resnet34_hist = resnet34.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10, verbose=2)
end = time()
print("===================================")
print(f'10 epochs in {(end-start)/60} min')

In [ ]:
hist_df = pd.DataFrame(resnet34_hist.history)
hist_json_file = 'ResNet34_10ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [ ]:
pred = resnet34.evaluate(test_gen)

In [ ]:
report(resnet34)

## ResNet above 50

In [ ]:
def BN_identity_block(x, filter):
    x_skip = x
    
    act = 'relu'
    
    x = Conv2D(filters=filter, kernel_size=(1,1), padding='valid')(x)
    x = BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation(act)(x)
    
    x = Conv2D(filters=filter, kernel_size=(3,3), padding='same')(x)
    x = BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation(act)(x)
    
    x = Conv2D(filters=filter*4, kernel_size=(1,1), padding='valid')(x)
    x = BatchNormalization(axis=3)(x)
    
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation(act)(x)
    
    return x

def BN_convolutional_block(x, filter, s):
    x_skip = x
    
    act = 'relu'
    
    x = Conv2D(filters=filter, kernel_size=(1,1), padding='valid', strides=s)(x)
    x = BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation(act)(x)
    
    x = Conv2D(filters=filter, kernel_size=(3,3), padding='same')(x)
    x = BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation(act)(x)
    
    x = Conv2D(filters=filter*4, kernel_size=(1,1), padding='valid')(x)
    x = BatchNormalization(axis=3)(x)
    
    x_skip = Conv2D(filters=filter*4, kernel_size=1, strides=s, padding='valid')(x_skip)
    
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation(act)(x)
    return x

In [ ]:
def BN_ResNet(input_shape=(256,256,3)):
    
    filters = 64
    act = 'relu'
    
    x_input = Input(shape=input_shape)
    
    x = Conv2D(filters=filters, kernel_size=(7,7), strides=2)(x_input)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation(act)(x)
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    
    block_layers = [3,4,6,3]
    for i in range(4):
        if i == 0:
            x = BN_convolutional_block(x=x, filter=filters, s=1)
            for j in range(block_layers[i]-1):
                x = BN_identity_block(x, filters)
        else:
            filters = filters*2
            x = BN_convolutional_block(x, filters, s=2)
            for j in range(block_layers[i] - 1):
                x = BN_identity_block(x, filters)
    
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dropout(0.4)(x)
    output = Dense(units=3, activation='softmax')(x)
    
    model = Model(inputs=x_input, outputs=output, name='ResNet50')
    
    return model

In [ ]:
resnet50 = BN_ResNet(input_shape=(256,256,3))
resnet50.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,auc_pr])

In [ ]:
resnet50.count_params()

In [ ]:
start = time()
resnet50_hist = resnet50.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10, verbose=2)
end = time()
print("===================================")
print(f'10 epochs in {(end-start)/60} min')

In [ ]:
hist_df = pd.DataFrame(resnet50_hist.history)
hist_json_file = 'ResNet50_10ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [ ]:
pred = resnet50.evaluate(test_gen)

In [ ]:
report(resnet50)

## DenseNet

In [ ]:
def convolutional_layer(x, filters, kernel=1, strides=1):
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = Conv2D(filters, kernel_size=kernel, strides=strides, padding='same')(x)
    return x

def dense_block(x, layers, filters):
    for _ in range(layers):
        x_d = convolutional_layer(x, 4*filters)
        x_d = convolutional_layer(x_d, filters, kernel=3)
        x = tf.keras.layers.Concatenate()([x_d,x])
    return x

def transition_layer(x):
    x = convolutional_layer(x, filters=x.shape[-1]//2)
    x = tf.keras.layers.AveragePooling2D(2, strides=2, padding='same')(x)
    return x

In [ ]:
def denseNet121(input_shape):
    input = Input(shape=input_shape)
    
    x = Conv2D(filters=64, kernel_size=7, strides=2, padding='same')(input)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = MaxPool2D(3, strides=2, padding='same')(x)
    
    for l in [6,12,24,16]:
        d = dense_block(x, layers=l, filters=32)
        x = Dropout(0.2)(x)
        x = transition_layer(d)
    x = Dropout(0.5)(x)
        
    x = tf.keras.layers.GlobalAveragePooling2D()(d)
    x = Flatten()(x)
    output = Dense(units=3, activation='softmax')(x)
    
    model = Model(inputs=input, outputs=output, name='DenseNet')
    
    return model

In [ ]:
densenet = denseNet121(input_shape=(256,256,3))
densenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[cat_acc,auc_roc,auc_pr])

In [ ]:
densenet.count_params()

In [ ]:
start = time()
densenet_hist = densenet.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10, verbose=2)
end = time()
print("===================================")
print(f'10 epochs in {(end-start)/60} min')

In [ ]:
hist_df = pd.DataFrame(densenet_hist.history)
hist_json_file = 'densenet_10ep_histori.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [ ]:
pred = densenet.evaluate(test_gen)

In [ ]:
report(densenet)

## Inception v1

In [ ]:
def inception_module(x,filters_1x1,filters_3x3_reduce,filters_3x3,
    filters_5x5_reduce,filters_5x5,filters_pool_proj,name=None):

    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu')(x)

    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu')(conv_3x3)
    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu')(conv_5x5)
    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu')(pool_proj)
    output = tf.keras.layers.Concatenate(axis=3, name=name)([conv_1x1, conv_3x3, conv_5x5, pool_proj])

    return output

In [ ]:
def inception_v1(input_shape=(256,256,3)):
    
    input = Input(shape=input_shape)
    
    x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2')(input)
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
    x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
    x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)
    
    x = inception_module(x,filters_1x1=64,filters_3x3_reduce=96,
     filters_3x3=128,filters_5x5_reduce=16,filters_5x5=32,filters_pool_proj=32,name='inception_3a')
    x = Dropout(0.2)(x)
    
    x = inception_module(x,filters_1x1=128,filters_3x3_reduce=128,
     filters_3x3=192,filters_5x5_reduce=32,filters_5x5=96,filters_pool_proj=64,name='inception_3b')
    x = Dropout(0.2)(x)
    
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)
    x = inception_module(x,filters_1x1=192,filters_3x3_reduce=96,
    filters_3x3=208,filters_5x5_reduce=16,filters_5x5=48,filters_pool_proj=64, name='inception_4a')
    x = Dropout(0.2)(x)
    
    x1 = tf.keras.layers.AveragePooling2D((5, 5), strides=3)(x)
    x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
    x1 = Flatten()(x1)
    x1 = Dense(1024, activation='relu')(x1)
    x1 = Dropout(0.7)(x1)
    x1 = Dense(3, activation='softmax', name='auxilliary_output_1')(x1)
    
    x = inception_module(x,filters_1x1=160,filters_3x3_reduce=112,
    filters_3x3=224,filters_5x5_reduce=24,filters_5x5=64,filters_pool_proj=64,name='inception_4b')
    x = Dropout(0.2)(x)
    
    x = inception_module(x,filters_1x1=128,filters_3x3_reduce=128,
     filters_3x3=256,filters_5x5_reduce=24,filters_5x5=64,filters_pool_proj=64,name='inception_4c')
    x = Dropout(0.2)(x)
    
    x = inception_module(x,filters_1x1=112,filters_3x3_reduce=144,
     filters_3x3=288,filters_5x5_reduce=32,filters_5x5=64,filters_pool_proj=64,name='inception_4d')
    x = Dropout(0.2)(x)
    
    x2 = tf.keras.layers.AveragePooling2D((5, 5), strides=3)(x)
    x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
    x2 = Flatten()(x2)
    x2 = Dense(1024, activation='relu')(x2)
    x2 = Dropout(0.7)(x2)
    x2 = Dense(3, activation='softmax', name='auxilliary_output_2')(x2)
    
    x = inception_module(x,filters_1x1=256,filters_3x3_reduce=160,
     filters_3x3=320,filters_5x5_reduce=32,filters_5x5=128,filters_pool_proj=128,name='inception_4e')
    
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)
    x = inception_module(x,filters_1x1=256,filters_3x3_reduce=160,
     filters_3x3=320,filters_5x5_reduce=32,filters_5x5=128,filters_pool_proj=128,name='inception_5a')
    
    x = inception_module(x, filters_1x1=384,filters_3x3_reduce=192,
     filters_3x3=384,filters_5x5_reduce=48,filters_5x5=128,filters_pool_proj=128,name='inception_5b')
    
    x = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)
    x = Dropout(0.4)(x)
    x = Dense(3, activation='softmax', name='output')(x)
    
    model = Model(inputs=input, outputs=[x, x1, x2], name='inception_v1')
    
    return model

In [ ]:
inception1 = inception_v1(input_shape=(256,256,3))
inception1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=['categorical_crossentropy','categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[1.0,0.3,0.3], metrics=[cat_acc,auc_roc,auc_pr])

In [ ]:
start = time()
inception1_hist = inception1.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=50, verbose=2)
end = time()
print("===================================")
print(f'50 epochs in {(end-start)/60} min')

In [ ]:
inception1.save_weights('Inception1.hdf5')

In [ ]:
hist_df = pd.DataFrame(inception1_hist.history)
hist_json_file = 'inception1_10ep_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [ ]:
pred = inception1.evaluate(test_gen)

In [ ]:
to_pred,label = test_gen.next()
inception1.predict(to_pred), label

In [ ]:
report(inception1)